# **Implementación de un Algoritmo Genético**

**Evaluation Exercise**

---

Implementad un algoritmo genético que obtenga las dimensiones de una piscina rectangular, cuyo volumen es 2500 m3, para que sea recubierta por la menor cantidad de azulejos posibles. Se debe tener en cuenta:
* La superficie de cada azulejo es 0.5 m2.
* El volumen de la piscina puede variar en +/-1%. 
* La longitud máxima de cada lado de la piscina es de 150m.
* La profundidad máxima es de 60m.

## Librerías

In [1]:
import random
import copy
import multiprocessing 
import numpy

## Creación de la población inicial

In [2]:
def build_initial_population(pop_size, ind_length):
    return [build_individual(ind_length) for j in range(pop_size)]

  #### Inicialización aleatoria de un individuo (cadena binaria)

In [3]:
def build_individual(length):
    ind = {}
    ind['chromosome'] = [random.randint(1, max_long), random.randint(1, max_long), random.randint(1, max_prof)]
    ind['fitness'] = None
    return ind

## Evaluación del individuo

In [4]:
def evaluate(pop):
    for ind in pop:
        ind['fitness'] = fitness(ind)

#### Fitness function

In [5]:
def fitness(ind):
   c_vol = constraint_volume(ind)
   c_dim = constraint_dimensions(ind)
   c = c_vol + c_dim

   superficie = 2*(ind['chromosome'][0] * ind['chromosome'][2]) + 2*(ind['chromosome'][1] * ind['chromosome'][2]) + (ind['chromosome'][0] * ind['chromosome'][1])

   if c > 0:
      return c + (2*(max_long * max_prof) + 2*(max_long * max_prof) + (max_long * max_long))/surf_tile
   
   return superficie / surf_tile

In [6]:
def constraint_volume(ind):
    vol = ind['chromosome'][0] * ind['chromosome'][1] * ind['chromosome'][2]
    if(volume*0.99 <= vol <= volume*1.01):
        return 0
    else:
        return abs(vol - volume)

In [7]:
def constraint_dimensions(ind):
    dist = 0
    if (ind['chromosome'][0] > max_long):
        dist += ind['chromosome'][0] - max_long
    if (ind['chromosome'][1] > max_long):
        dist += ind['chromosome'][1] - max_long
    if (ind['chromosome'][2] > max_prof):
        dist += ind['chromosome'][2] - max_prof

    for i in range(0,3):
        if (ind['chromosome'][i] < 0):
            dist += abs(ind['chromosome'][i])
    
    return dist

## Tournament selection

In [8]:
def tournament(population, pop_size, k_tournament, optimization):
    parents_selected = []
    #Seleccionar padres hasta tener el mismo número de individuos que en la población
    for i in range(0, pop_size):
        parents_selected.append(random_selection(population, k_tournament, optimization))
    return parents_selected

#### Selección aleatoria de los mejores individuos

In [9]:
def random_selection(population, k_tournament, optimization):
    best = None
    #Seleccionar k_tournament individuos aleatoriamente
    size = int(len(population) * k_tournament)
    selected = []
    for e in range(0, size):
        i = random.randint(0, len(population) - 1)
        selected.append(population[i])
    #Seleccionar el mejor del conjunto aleatorio
    if optimization == 'MIN':
        best = list(sorted(selected, reverse=False, key=lambda ind: ind['fitness']))[0]
    else:
        best = list(sorted(selected, reverse=True, key=lambda ind: ind['fitness']))[0]
    return best.copy()

## Crossover

#### Two-point crossover

In [10]:
def two_point_crossover(father, mother):
    index1 = random.randint(1, len(father['chromosome']) - 2)
    index2 = random.randint(1, len(father['chromosome']) - 2)
    if index1 > index2: index1, index2 = index2, index1
    child1 = {}
    child2 = {}
    child1['chromosome'] = father['chromosome'][:index1] + mother['chromosome'][index1:index2] + father['chromosome'][index2:]
    child2['chromosome'] = mother['chromosome'][:index1] + father['chromosome'][index1:index2] + mother['chromosome'][index2:]
    child1['fitness'] = None
    child2['fitness'] = None
    return copy.deepcopy(child1), copy.deepcopy(child2)

## Mutation

### Uniform mutation

In [11]:
def uniform_mutation(individual, prob_mutation):
    for i in range(0, len(individual['chromosome'])):
        mutate = random.random() < prob_mutation
        if mutate:
            vary = random.randint(1, max_long)
            individual['chromosome'][i] = vary
    return individual

### Gaussian mutation

In [12]:
def gaussian_mutation(individual, prob_mutation, std=10):
    for i in range(0, len(individual['chromosome'])):
        mutate = random.random() < prob_mutation
        if mutate:
            value = int(numpy.random.normal(individual['chromosome'][i], std, 1)[0])
            individual['chromosome'][i] = value
    return individual

## Remplazo de la población

#### Elitismo de la mejor solución

In [13]:
def update_population(population, new_population, optimization):
    if optimization == 'MIN':
        new_population = sorted(new_population, reverse=False, key=lambda ind: ind['fitness'])
        population = sorted(population, reverse=False, key=lambda ind: ind['fitness'])
    else:
        new_population = sorted(new_population, reverse=True, key=lambda ind: ind['fitness'])
        population = sorted(population, reverse=True, key=lambda ind: ind['fitness'])
    new_population[-1] = population[0]
    
    return new_population

## Construcción del proceso genético

In [14]:
#Parámetros
pop_size = 100
ind_length = 3
max_generations = 200
k_tournament = 0.3
optimization = 'MIN'
prob_crossover = 0.9
prob_mutation = 1 / ind_length

global volume, surf_tile, max_long, max_prof
volume = 2500 #[m3] Volumen piscina
surf_tile = 0.5 #[m2] Superficie del azulejo
max_long = 150 #[m] Máxima longitud de cada lado
max_prof = 60 #[m] Máxima profundidad

#Inicialización de la población
population = build_initial_population(pop_size, ind_length)

#Evaluación de los individuos
evaluate(population)

#Bucle genético (criterio de finalización)
for generation in range(0, max_generations):
    #Selección de padres
    parents_selected = tournament(population, pop_size, k_tournament, optimization)
    #Cruce de los padres
    i = 0
    new_population = []
    while i < len(parents_selected):
        if random.random() < prob_crossover:
            child1, child2 = two_point_crossover(parents_selected[i], parents_selected[i+1])
        else:
            child1 = copy.deepcopy(parents_selected[i])
            child2 = copy.deepcopy(parents_selected[i+1])
        new_population.append(child1)
        new_population.append(child2)
        i = i + 2
    #Mutación de los hijos
    for ch in new_population:
        #Elegir tipo de mutación
        uniform_mutation(ch, prob_mutation)
        #gaussian_mutation(ch, prob_mutation, std=5)
    #Evaluación de los hijos
    evaluate(new_population)
    #Remplazar la población
    population = update_population(population, new_population, optimization)          
    if optimization == 'MIN':
        best = list(sorted(population, reverse=False, key=lambda ind: ind['fitness']))[0]
    else:
        best = list(sorted(population, reverse=True, key=lambda ind: ind['fitness']))[0]
    print("Generación", generation, ": fitness", best['fitness'], " - ", best['chromosome'])

Generación 0 : fitness 117074.0  -  [13, 99, 2]
Generación 1 : fitness 3402.0  -  [13, 97, 2]
Generación 2 : fitness 3402.0  -  [13, 97, 2]
Generación 3 : fitness 3402.0  -  [13, 97, 2]
Generación 4 : fitness 3368.0  -  [13, 96, 2]
Generación 5 : fitness 3368.0  -  [13, 96, 2]
Generación 6 : fitness 3368.0  -  [13, 96, 2]
Generación 7 : fitness 3368.0  -  [13, 96, 2]
Generación 8 : fitness 3368.0  -  [13, 96, 2]
Generación 9 : fitness 3368.0  -  [13, 96, 2]
Generación 10 : fitness 3368.0  -  [13, 96, 2]
Generación 11 : fitness 3368.0  -  [13, 96, 2]
Generación 12 : fitness 3368.0  -  [13, 96, 2]
Generación 13 : fitness 3368.0  -  [13, 96, 2]
Generación 14 : fitness 3368.0  -  [13, 96, 2]
Generación 15 : fitness 3368.0  -  [13, 96, 2]
Generación 16 : fitness 3368.0  -  [13, 96, 2]
Generación 17 : fitness 3368.0  -  [13, 96, 2]
Generación 18 : fitness 3368.0  -  [13, 96, 2]
Generación 19 : fitness 3368.0  -  [13, 96, 2]


Generación 20 : fitness 3368.0  -  [13, 96, 2]
Generación 21 : fitness 3368.0  -  [13, 96, 2]
Generación 22 : fitness 3368.0  -  [13, 96, 2]
Generación 23 : fitness 3368.0  -  [13, 96, 2]
Generación 24 : fitness 3084.0  -  [34, 37, 2]
Generación 25 : fitness 3084.0  -  [34, 37, 2]
Generación 26 : fitness 3084.0  -  [34, 37, 2]
Generación 27 : fitness 3084.0  -  [34, 37, 2]
Generación 28 : fitness 3084.0  -  [34, 37, 2]
Generación 29 : fitness 3084.0  -  [34, 37, 2]
Generación 30 : fitness 3084.0  -  [34, 37, 2]
Generación 31 : fitness 3084.0  -  [34, 37, 2]
Generación 32 : fitness 3084.0  -  [34, 37, 2]
Generación 33 : fitness 3084.0  -  [34, 37, 2]
Generación 34 : fitness 3084.0  -  [34, 37, 2]
Generación 35 : fitness 3084.0  -  [34, 37, 2]
Generación 36 : fitness 3084.0  -  [34, 37, 2]
Generación 37 : fitness 3084.0  -  [34, 37, 2]
Generación 38 : fitness 3084.0  -  [34, 37, 2]
Generación 39 : fitness 3084.0  -  [34, 37, 2]
Generación 40 : fitness 3084.0  -  [34, 37, 2]
Generación 41